# 라이브러리 준비 및 TFRecord 불러오기

In [1]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score

In [2]:
import os, glob
import logging

from pprint import pprint
from tqdm import tqdm
import json
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅한다
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 필요 변수 및 하이퍼 파라미터를 미리 지정

In [3]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 512  # Maximum length of the input to the Encoder
MIN_TARGET_LENGTH = 5  # Minimum length of the output by Decoder
MAX_TARGET_LENGTH = 128  # Maximum length of the output by Decoder
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# HF model hub에서 가져올 모델 이름
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

## TFRecords 불러오기

In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
def _parse_function(example_proto):

    name_to_features = {'id': tf.io.FixedLenFeature([], tf.string),
                        'document': tf.io.FixedLenFeature([], tf.string),
                        'summary': tf.io.FixedLenFeature([], tf.string)}

    example = tf.io.parse_single_example(example_proto, name_to_features)

    for name in list(example.keys()):
        t = example[name]
        t = tf.cast(t, tf.string)
        example[name] = t

    return example

def load_tfrecord_dataset(tfrecord_name, batch_size, shuffle=True, buffer_size=10240):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()

    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)

    name_to_features = {'id': tf.io.FixedLenFeature([], tf.string),
                        'document': tf.io.FixedLenFeature([], tf.string),
                        'summary': tf.io.FixedLenFeature([], tf.string)}


    dataset = raw_dataset.map(
        _parse_function,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )



    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [7]:
tfr_path = 'gs://ohsori-tfrecord/tfrecord/literature.tfrecords'
tf_record = load_tfrecord_dataset(tfr_path, batch_size=BATCH_SIZE, shuffle=False)
tf_record = iter(tf_record)

In [ ]:
list(tf_record)[0]

In [8]:
# 데이터 확인
[summary.decode('utf-8') for summary in next(tf_record)['summary'].numpy()]

['나는 남에게 제시할 이야기는 습득하지 못했지만 나의 고발정신을 지적했던 비판자들에 대한 회고까지 축방할 수는 없다.',
 '카페의 여급으로 있는 보배는 당시 불만과 마음의 가난 때문에 건을 구하였다.',
 '취린소와 기타 채권자들이 의인은 도와야 한다며 또다시 무일푼이 된 L씨의 토지를 처분하지 않고 그냥 두었다.',
 '과장은 그렇게 조바심을 내지 않아도 살 수 있다는 것을 알 법도 한데 여전히 삶에 급급해 하고 그로 인해 밑에 있는 젊은 사람들이 전전한다.',
 '시집이 친정보다 더 유족하니 친정이나 시집 어디든지 가서 몸과 마음을 가라앉히고 치산을 해야만 할 것이다.',
 '두 젊은이는 이 원정으로 영원한 우정을 맹세했고 테무 친은 그가 죽을 때까지 충실하게 일해 준 최초의 부하를 얻었으며 그들은 결정적 순간에 습격하는 기술 등을 배우기 시작했다.',
 '요사히 구미에서는 원색과 극렬한 색이 유행이라고 한다.',
 '지독한 피곤이 엄습한 두식이는 침대에 몸을 내어던지고 담배를 붙여 물었지만 다 태우지도 못한 채 내려뜨리고 잠이 들었다.']

# 데이터 전처리

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [10]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b", "psyche/KoT5-summarization"]:
    prefix = "요약: "
else:
    prefix = ""

## 전처리 함수 선언하기
- prefix 추가 및 tokenization
- token_type_ids, attention_mask 생성

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs